In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd /home/aditya/git/kaggle_pneumonia 
%env PROJECT_PATH = /home/aditya/git/kaggle_pneumonia 

/home/aditya/git/kaggle_pneumonia
env: PROJECT_PATH=/home/aditya/git/kaggle_pneumonia


In [3]:
%matplotlib inline
import seaborn as sns

In [4]:
import pandas as pd
import numpy as np
import pydicom
from PIL import Image
import multiprocessing as mp
from tqdm import tqdm_notebook as tqdm

In [5]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import skimage
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [7]:
from utils.envs import *
from utils.data_load import *
from utils.lr_finder import lr_plot, lr_find
from utils.common import get_batch_info
from utils.checkpoint import save_checkpoint, load_cp_model, load_cp_optim
from utils.logger import logger

In [8]:
train_label_df = pd.read_csv(train_label_repo)
train_label_df['area'] = train_label_df.width * train_label_df.height
single_label_df = train_label_df.sort_values('area', ascending = False).drop_duplicates('patientId').sort_index().reset_index(drop = True).copy()
single_label_df.fillna(0, inplace = True)

In [9]:
single_label_df['x2'] = single_label_df['x'] + single_label_df['width']
single_label_df['y2'] = single_label_df['y'] + single_label_df['height']

In [10]:
def to_string(row_val):
    x, y, x2, y2 = row_val
    return "{} {} {} {}".format(int(x), int(y), int(x2), int(y2))

In [11]:
single_label_df['bb_label'] = single_label_df[['x', 'y', 'x2', 'y2']].apply(to_string, axis = 1)

In [12]:
single_label_df['true_path'] = single_label_df['patientId'].apply(lambda x : "train_images/{}.png".format(x))

In [13]:
bb_df = single_label_df[single_label_df.Target == 1][['patientId', 'bb_label']].copy()
label_df = single_label_df[['patientId', 'Target']].copy()
bb_df.columns = ['name', 'label']
label_df.columns = ['name', 'label']

In [14]:
bb_df.to_csv(bb_repo, index = False)
label_df.to_csv(label_repo, index = False)

In [15]:
image_id = [x for x in single_label_df.patientId.values]

In [16]:
def store_train_image(patientId):
    filename = patientId + '.dcm'
    outname = patientId + '.png'
    filepath = os.path.join(train_dcm_path, filename)
    outpath = os.path.join(train_images_path, outname)
    img_array = pydicom.read_file(filepath).pixel_array
    img_array = skimage.color.gray2rgb(img_array)
    img_array = skimage.transform.resize(img_array, (224, 224))
    skimage.io.imsave(outpath, img_array)

In [21]:
def store_test_image(patientId):
    filename = patientId + '.dcm'
    outname = patientId + '.png'
    filepath = os.path.join(test_dcm_path, filename)
    outpath = os.path.join(test_images_path, outname)
    img_array = pydicom.read_file(filepath).pixel_array
    img_array = skimage.color.gray2rgb(img_array)
    img_array = skimage.transform.resize(img_array, (224, 224))
    skimage.io.imsave(outpath, img_array)

In [19]:
test_images=[ x.split('.')[0] for x in os.listdir(test_dcm_path) if x.endswith('dcm')]

In [22]:
import multiprocessing
from multiprocessing import Pool

p = Pool(12)
p.map(store_test_image, test_images)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [ ]:
import multiprocessing
from multiprocessing import Pool

p = Pool(12)
p.map(store_train_image, image_id)